In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pathlib

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Get data

In [2]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url, extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [3]:
# check data
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [4]:
# simplify the target for binary classfication
dataframe['target'] = np.where(dataframe['AdoptionSpeed'] == 4, 0, 1)  # original dataset 4 is not adopted
# remove unused columns
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

# Preprocess data

In [5]:
# split sets
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'val examples')
print(len(test), 'test examples')

7383 train examples
1846 val examples
2308 test examples


In [6]:
# create dataset from dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()

    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [7]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [8]:
# check dataset
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of labels:', label_batch)

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 2 24 12  8  2], shape=(5,), dtype=int64)
A batch of labels: tf.Tensor([0 1 0 1 1], shape=(5,), dtype=int32)


### preprocessing layers

In [9]:
# normalize numeric features
def get_normalization_layer(name, dataset):
    normalizer = preprocessing.Normalization()

    # create a dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

    normalizer.adapt(feature_ds)
    return normalizer

In [10]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.51509947],
       [-0.84064126],
       [-0.84064126],
       [-0.84064126],
       [ 1.1126094 ]], dtype=float32)>

In [11]:
# one hot encode categorical features
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # StringLookup layer which turns strings to integer indices
    if dtype == 'string':
        index = preprocessing.StringLookup(max_tokens=max_tokens)
    else:
        index = preprocessing.IntegerLookup(max_values=max_tokens)

    # create a dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

    # learn the set of possible values and assign them a fixed integer index
    index.adapt(feature_ds)

    # create Discretization for integer indices
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

    # create a dataset that only yields our feature
    feature_ds = feature_ds.map(index)

    encoder.adapt(feature_ds)

    # one hot encode
    return lambda feature: encoder(index(feature))

In [12]:
# encode the type feature
type_col = train_features['Type']
layer = get_category_encoding_layer('Type', train_ds, 'string')
layer(type_col)

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)>

In [13]:
# can also encode integer features
type_col = train_features['Age']
category_encoding_layer = get_category_encoding_layer('Age', train_ds, 'int64', 5)
category_encoding_layer(type_col)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

# Choose features

In [14]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [15]:
all_inputs = []
encoded_features = []

# numeric features
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

In [16]:
# categorical features of integers
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')
encoding_layer = get_category_encoding_layer('Age', train_ds, dtype='int64', max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

In [17]:
# categorical features of strings
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']

for header in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string', max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

# Build model

In [18]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation='relu')(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])


In [19]:
# show connectivity
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [20]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 0.6064 - accuracy: 0.6366 - val_loss: 0.5621 - val_accuracy: 0.7232
Epoch 2/10
29/29 [==============================] - 0s 6ms/step - loss: 0.5833 - accuracy: 0.6721 - val_loss: 0.5449 - val_accuracy: 0.7216
Epoch 3/10
29/29 [==============================] - 0s 7ms/step - loss: 0.5597 - accuracy: 0.6908 - val_loss: 0.5355 - val_accuracy: 0.7221
Epoch 4/10
29/29 [==============================] - 0s 7ms/step - loss: 0.5535 - accuracy: 0.7004 - val_loss: 0.5277 - val_accuracy: 0.7221
Epoch 5/10
29/29 [==============================] - 0s 7ms/step - loss: 0.5464 - accuracy: 0.6984 - val_loss: 0.5224 - val_accuracy: 0.7302
Epoch 6/10
29/29 [==============================] - 0s 7ms/step - loss: 0.5386 - accuracy: 0.7123 - val_loss: 0.5182 - val_accuracy: 0.7329
Epoch 7/10
29/29 [==============================] - 0s 6ms/step - loss: 0.5328 - accuracy: 0.7154 - val_loss: 0.5157 - val_accuracy: 0.7313
Epoch 8/10
29/29 [=

In [21]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

10/10 [==============================] - 0s 4ms/step - loss: 0.5132 - accuracy: 0.7305
Accuracy 0.7305026054382324


# Inference

In [22]:
model.save('my_pet_classifier')
reloaded_model = tf.keras.models.load_model('my_pet_classifier')

INFO:tensorflow:Assets written to: my_pet_classifier\assets


In [23]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

This particular pet had a 84.6 percent probability of getting adopted.
